### In this notebook, we demonstrate how to visulaize embeddings with tenosrboard 

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector

In [2]:
class TF_visualizer(object):
    def __init__(self, dimension, vecs_file, metadata_file, output_path):
        self.dimension = dimension
        self.vecs_file = vecs_file
        self.metadata_file = metadata_file
        self.output_path = output_path
        
        self.vecs = []
        embeddings = np.load(vecs_file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')      
        for i in range(len(embeddings)):
                self.vecs.append(embeddings[i])
                
    def visualize(self):
        # adding into projector
        config = projector.ProjectorConfig()

        placeholder = np.zeros((len(self.vecs), self.dimension))
        for i, line in enumerate( self.vecs ):   
             placeholder[i] = line
        embedding_var = tf.Variable(placeholder, trainable=False, name='metadata')

        embed = config.embeddings.add()
        embed.tensor_name = embedding_var.name
        embed.metadata_path = self.metadata_file

        # define the model without training
        sess = tf.compat.v1.InteractiveSession()
        with tf.compat.v1.Session() as sess:
            init = tf.compat.v1.global_variables_initializer()
            sess.run(init)
        saver = tf.compat.v1.train.Saver([embedding_var])
        
        saver.save(sess, os.path.join(self.output_path, 'w2x_metadata.ckpt'))

        writer = tf.summary.create_file_writer(self.output_path)
        projector.visualize_embeddings(self.output_path, config)
        sess.close()
        print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(self.output_path))


In [5]:
output = '/Users/ravanv/PycharmProjects/KCET/notebooks/data/embeddings_final' #change the path according to the path to your data

# create a new tensor board visualizer
visualizer = TF_visualizer(dimension = 100,
                           vecs_file =  os.path.join(output, "embedding_SG_dim100_upto2020.npy"),
                           metadata_file = os.path.join(output, "words_SG_upto2020.txt"),
                           output_path = output)
visualizer.visualize()


Run `tensorboard --logdir=/Users/ravanv/PycharmProjects/KCET/notebooks/data/embeddings_final` to run visualize result on tensorboard
